In [1]:
import sys
sys.path.append('/root/autodl-tmp/One_Month_Paper')
import faiss
import pandas as pd
import numpy as np
import torch.cuda
import torch.backends
import time
import os
from connector.embedding.embed import bgeEmbeddings
import chromadb
from tqdm import tqdm

In [2]:
search_latency_fmt = "search latency = {:.4f}s"

In [3]:
# 加载 embedding 模型     
start_time = time.time()
embed_model = bgeEmbeddings(
    '/root/autodl-tmp/One_Month_Paper/model/bge-large-en-v1.5', 
    batch_size=64,
    max_len=512,
    device='cuda:0'
)
end_time = time.time()
print("load done embedding model")
print(search_latency_fmt.format(end_time - start_time))

successful load embedding model
load done embedding model
search latency = 1.2061s


In [4]:
# 读取数据
print("开始读取数据......")
# csv_df = pd.read_csv("datas/train.csv")  # 读取训练数据
# csv_data = csv_df.loc[:15]
csv_data = pd.read_csv('/root/autodl-tmp/One_Month_Paper/connector/vectorstore/data/merged_data.csv')  # 读取训练数据
print(csv_data.shape)  # (476066, 3)

num_entities = len(csv_data)
print(num_entities)
print(csv_data["Abstraction"][0])


开始读取数据......
(8069, 6)
8069
The paper considers a non-reductionist theory of consciousness, which is not reducible to theories of reality and to physiological or psychological theories. Following D.I.Dubrovsky's "informational approach" to the "Mind-Brain Problem", we consider the reality through the prism of information about observed phenomena, which, in turn, is perceived by subjective reality through sensations, perceptions, feelings, etc., which, in turn, are information about the corresponding brain processes. Within this framework the following principle of the Information Theory of Consciousness (ITS) development is put forward: the brain discovers all possible causal relations in the external world and makes all possible inferences by them. The paper shows that ITS built on this principle: (1) also base on the information laws of the structure of external world; (2) explains the structure and functioning of the brain functional systems and cellular ensembles; (3) ensures maxim

In [5]:
print("开始使用向量embedding模型嵌入向量数据......")
now_vectors = []
start_time = time.time()
for index,row in tqdm(csv_data.iterrows(), total=num_entities, desc="Processing DataFrame"):
    now_vectors.append(embed_model.embed_documents([row['Abstraction']])[0])
end_time = time.time()
print(search_latency_fmt.format(end_time - start_time))

开始使用向量embedding模型嵌入向量数据......


Processing DataFrame:  35%|███▌      | 2829/8069 [00:59<01:50, 47.59it/s]


KeyboardInterrupt: 

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, learning_rate=200, metric='cosine',n_jobs=-1)
tsne.fit_transform(embs)
outs_2d = np.array(tsne.embedding_)

In [48]:
print(type(now_vectors))
print(now_vectors.dtype)
now_vectors = now_vectors.astype('float32')
# print(len(now_vectors[0]))
# print(len(now_vectors))
# now_vectors = np.array(now_vectors)
# print(now_vectors.size)
print("开始插入向量数据......")
dimension = now_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)

# index = faiss.IndexHNSWFlat(dimension,4)

# nlist = 50
# quantizer = faiss.IndexFlatL2(dimension)
# index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
# index.train(now_vectors)

start_time = time.time()
# index.train(now_vectors)
index.add(now_vectors)
end_time = time.time()
print(search_latency_fmt.format(end_time - start_time))
print("查看索引数据：")
print(index.ntotal)



<class 'numpy.ndarray'>
float32
开始插入向量数据......
search latency = 0.0113s
查看索引数据：
8069


In [49]:
# 保存
dump_file = 'vectors_file/faiss_all.index'
faiss.write_index(index, dump_file)


# print("开始查询")
# while True:
#     search_sen = input("请输入查询语句：")
#     search_vec = np.array([embed_model.embed_query(search_sen)])
#     search_vec = search_vec.astype('float32')
#     topK = 7
#     start_time = time.time()
#     D, I = index.search(search_vec, topK)
#     end_time = time.time()
#     # result = csv_data["sentences"].iloc[I[0]]
#     # print(result)
#     print(search_latency_fmt.format(end_time - start_time))

# 单独加载
index = faiss.read_index('vectors_file/faiss_all.index')
search_sen = " I wnat to know something about RAG ! "
start_time = time.time()
search_vec = np.array([embed_model.embed_query(search_sen)]).astype('float32')
end_time = time.time()
print(search_latency_fmt.format(end_time - start_time))
#     topK = 7
# search_vec = np.array(search_vec)
topK = 7
start_time = time.time()
D, I = index.search(search_vec, topK)
end_time = time.time()
print(D)
print(I)
print(search_latency_fmt.format(end_time - start_time))

# '''向量规模：476066；向量插入速度：1s，平均查询速度：130ms
# 请输入查询语句：
# 0         是的，我想一个洞穴也会有这样的问题
# 1          我认为洞穴可能会有更严重的问题。
# 110556       在这些洞穴里事情可能会变糟。
# Name: sentences, dtype: object
# search latency = 0.1315s
# '''

search latency = 0.0265s
[[0.59140813 0.59934604 0.60930794 0.62627405 0.6381023  0.64467156
  0.6520666 ]]
[[2541 2407 7574 2178  916 5432 2459]]
search latency = 0.0034s
